In [1]:
import pandas as pd

# Load protein sequences
sequences_df = pd.read_csv('pdb_sequences_success.csv')

# Create a dictionary mapping from (pdb_id, chain_id) to sequence
sequence_dict = {}
for index, row in sequences_df.iterrows():
    key = (row['pdb_id'], row['chain_id'])
    sequence = row['sequence']
    sequence_dict[key] = sequence

# Function to check if a sequence contains only standard amino acids
def is_valid_sequence(sequence, valid_amino_acids):
    return all(aa in valid_amino_acids for aa in sequence)

# Set of valid amino acids
# Include 'X' in the set of valid amino acids
valid_amino_acids = set('ACDEFGHIKLMNPQRSTVWYX')

# Filter the sequence dictionary
filtered_sequence_dict = {}

for key, sequence in sequence_dict.items():
    if is_valid_sequence(sequence, valid_amino_acids):
        filtered_sequence_dict[key] = sequence
    else:
        print(f"Sequence for PDB ID {key} contains unknown amino acids and will be excluded.")

In [2]:
# Collect all unique characters from all sequences
all_sequences = sequence_dict.values()
unique_chars = set(''.join(all_sequences))
print("Unique characters in sequences:", unique_chars)
print(len(unique_chars))

Unique characters in sequences: {'S', 'R', 'W', 'I', 'N', 'Q', 'E', 'L', 'T', 'A', 'X', 'F', 'M', 'C', 'Y', 'G', 'K', 'V', 'H', 'D', 'P'}
21


In [3]:
# Load interaction data
interaction_pairs = []
with open('interactions_data.txt', 'r') as file:
    for line in file:
        pdb_id1, pdb_id2, interaction = line.strip().split()
        interaction_pairs.append((pdb_id1, pdb_id2, int(interaction)))
interaction_pairs

[('3BLV', '4BSZ', 0),
 ('3ETU', '3JCK', 0),
 ('3C66', '2QRJ', 0),
 ('1S4U', '1S2M', 0),
 ('6BGT', '2V76', 1),
 ('5OQM', '4WWU', 0),
 ('2LQN', '4QSY', 1),
 ('1KL7', '2M6M', 1),
 ('2LCS', '1M2V', 0),
 ('3A4S', '4BKG', 1),
 ('2QLV', '5Y81', 0),
 ('1BXL', '5MGX', 1),
 ('4AYD', '1HAQ', 1),
 ('3N1G', '3N1G', 1),
 ('2XKX', '5M7M', 1),
 ('2AZE', '3EI4', 1),
 ('3JCM', '6EMK', 0),
 ('2DYM', '5CYZ', 0),
 ('2XUR', '3GLF', 1),
 ('2D1P', '2D1P', 1),
 ('4XH9', '4WYT', 1),
 ('2OEV', '2DQ7', 1),
 ('4WX8', '1FNT', 0),
 ('4BBN', '3ZYQ', 1),
 ('1WG0', '2DLC', 0),
 ('3VH2', '1SCG', 0),
 ('2PJW', '5T94', 0),
 ('2PJW', '4GFB', 0),
 ('5BW8', '3BLV', 0),
 ('4BY6', '6GEJ', 0),
 ('4WX8', '5WLC', 0),
 ('2VOF', '2L58', 1),
 ('2V1R', '4EQ6', 0),
 ('6GIQ', '5T7H', 0),
 ('4FTG', '4HRG', 1),
 ('5BWK', '3QT1', 0),
 ('4CY1', '5M23', 1),
 ('2MGS', '1M8A', 1),
 ('3MDY', '2BHK', 1),
 ('6DLO', '4AF3', 1),
 ('1WG0', '5GM6', 0),
 ('5TR4', '2IO1', 1),
 ('5GM6', '2L9B', 0),
 ('2VGE', '2VGE', 1),
 ('3ETU', '1S2M', 0),
 ('3NMZ', 

In [4]:
# Update filtered_sequence_dict to map pdb_id to sequence
sequence_dict_pdb_only = {}

for key, sequence in filtered_sequence_dict.items():
    pdb_id, chain_id = key
    if pdb_id not in sequence_dict_pdb_only:
        sequence_dict_pdb_only[pdb_id] = sequence
    else:
        if len(sequence) < len(sequence_dict_pdb_only[pdb_id]):
            sequence_dict_pdb_only[pdb_id] = sequence


sequence_dict_pdb_only


{'2P5Z': 'MSLEALMNVQFFDHAHHKLKIRGLQSPVDVLTFEGREQLSTPFRYDIQFTSSDKAIAPESVLMQDGAFSLTAPPVQGMPVQTALRTLHGVITGFKHLSSSQDEARYEVRLEPRMALLTRSRQNAIYQNQTVPQIVEKILRERHQMRGQDFVFNLKSEYPAREQVMQYGEDDLTFVSRLLSEVGIWFRFATDARLKIEVIEFYDDQSGYERGLTLPLRHPSGLFDGETEAVWGLNTAYSVVEKSVSTRDYNYREATAEMTTGQHDATGGDNTTYGEAYHYADNFLQQGDKEAAESGAFYARIRHERYLNEQAILKGQSTSSLLMPGLEIKVQGDDAPAVFRKGVLITGVTTSAARDRSYELTFTAIPYSERYGYRPALIPRPVMAGTLPARVTSTVKNDIYAHIDKDGRYRVNLDFDRDTWKPGYESLWVRQSRPYAGDTYGLHLPLLAGTEVSIAFEEGNPDRPYIAGVKHDSAHTDHVTIQNEGHHHHHH',
 '4ALI': 'MKHHHHHHPMSDYDIPTTENLYFQGAMVNLENKTYVIMGIANKRSIAFGVAKVLDQLGAKLVFTYRKERSRKELEKLLEQLNQPEAHLYQIDVQSDEEVINGFEQIGKDVGNIDGVYHSIAFANMEDLRGRFSETSREGFLLAQDISSYSLTIVAHEAKKLMPEGGSIVATTYLGGEFAVQNYNVMGVAKASLEANVKYLALDLGPDNIRVNAISAGPIRTLSAKGVGGFNTILKEIEERAPLKRNVDQVEVGKTAAYLLSDLSSGVTGENIHVDSGFHAIK',
 '3BG5': 'MGSSHHHHHHSSGLVPRGSHMASMKQIKKLLVANRGEIAIRIFRAAAELDISTVAIYSNEDKSSLHRYKADESYLVGSDLGPAESYLNIERIIDVAKQANVDAIHPGYGFLSENEQFARRCAEEGIKFIGPHLEHLDMFGDKVKARTTAIKADLPVIPGTDGPIKSYELAKEFAEEAGFPLMIKATSGGGG

In [5]:
# Filter pairs where both sequences are available and valid
filtered_pairs = []
missing_or_invalid_sequences = []

for pdb_id1, pdb_id2, interaction in interaction_pairs:
    if pdb_id1 in sequence_dict_pdb_only and pdb_id2 in sequence_dict_pdb_only:
        filtered_pairs.append((pdb_id1, pdb_id2, interaction))
    else:
        missing_or_invalid_sequences.append((pdb_id1, pdb_id2))

# Count the number of interactions with label 1 and label 0
interaction_counts = {0: 0, 1: 0}

for _, _, interaction in filtered_pairs:
    if interaction == 0:
        interaction_counts[0] += 1
    elif interaction == 1:
        interaction_counts[1] += 1
    else:
        print(f"Unexpected interaction label: {interaction}")

print(f"Number of interaction pairs with interaction 1: {interaction_counts[1]}")
print(f"Number of interaction pairs with interaction 0: {interaction_counts[0]}")


print(f"Total pairs after filtering: {len(filtered_pairs)}")
print(f"Pairs excluded due to missing or invalid sequences: {len(missing_or_invalid_sequences)}")


Number of interaction pairs with interaction 1: 4918
Number of interaction pairs with interaction 0: 4888
Total pairs after filtering: 9806
Pairs excluded due to missing or invalid sequences: 198


In [6]:
import numpy as np

# Standard amino acids, including 'X' for unknowns
amino_acids = 'ACDEFGHIKLMNPQRSTVWYX'

# Create amino acid to integer mapping, starting from 1 to reserve 0 for padding
aa_to_int = {aa: idx + 1 for idx, aa in enumerate(amino_acids)}
aa_to_int['<PAD>'] = 0  # Add a padding token mapped to 0

# Reverse mapping from integers to amino acids
int_to_aa = {idx: aa for aa, idx in aa_to_int.items()}

def integer_encode_sequence(sequence, max_length):
    # Map amino acids to integers
    int_sequence = [aa_to_int.get(aa, aa_to_int['X']) for aa in sequence[:max_length]]
    # Pad with zeros if sequence is shorter than max_length
    padding_length = max_length - len(int_sequence)
    if padding_length > 0:
        int_sequence += [aa_to_int['<PAD>']] * padding_length  # 0 is the padding index
    return int_sequence

# Determine the maximum sequence length
sequence_lengths = [len(seq) for seq in sequence_dict_pdb_only.values()]
max_sequence_length = max(sequence_lengths)

# Alternatively, set a fixed max length to limit the input size
# For example, set max_sequence_length = 1000
# max_sequence_length = 1000

X = []
y = []

for pdb_id1, pdb_id2, interaction in filtered_pairs:
    seq1 = sequence_dict_pdb_only[pdb_id1]
    seq2 = sequence_dict_pdb_only[pdb_id2]
    
    # Integer encode sequences
    int_seq1 = integer_encode_sequence(seq1, max_sequence_length)
    int_seq2 = integer_encode_sequence(seq2, max_sequence_length)
    
    # Combine the two sequences
    combined_sequence = int_seq1 + int_seq2  # Concatenate the integer sequences
    
    X.append(combined_sequence)
    y.append(interaction)

X = np.array(X)
y = np.array(y)


In [7]:
from sklearn.model_selection import train_test_split

# Split the data into train, dev, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)  # 80% train
X_dev, X_test, y_dev, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 10% dev, 10% test

print(f"Training examples: {len(X_train)}")
print(f"Validation examples: {len(X_dev)}")
print(f"Test examples: {len(X_test)}")


Training examples: 7844
Validation examples: 981
Test examples: 981


In [8]:
# Flatten the input sequences for feed-forward neural network
X_train_flat = X_train.reshape((X_train.shape[0], -1))
X_dev_flat = X_dev.reshape((X_dev.shape[0], -1))
X_test_flat = X_test.reshape((X_test.shape[0], -1))

print(f"Input shape: {X_train_flat.shape}")


Input shape: (7844, 9292)


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

input_dim = X_train_flat.shape[1]

model = Sequential()
model.add(Dense(512, activation='relu', input_dim=input_dim))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


2024-11-15 16:46:52.214551: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 16:46:52.369743: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731718012.444684 2401937 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731718012.461297 2401937 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 16:46:52.604096: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │     4,758,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,889,601 (18.65 MB)

 Trainable params: 4,889,601 (18.65 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Train the model
history = model.fit(
    X_train_flat, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_dev_flat, y_dev)
)


Epoch 1/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.5327 - loss: 2.7893 - val_accuracy: 0.6004 - val_loss: 0.7349
Epoch 2/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5948 - loss: 0.7917 - val_accuracy: 0.7288 - val_loss: 0.6070
Epoch 3/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5892 - loss: 0.6732 - val_accuracy: 0.7299 - val_loss: 0.5238
Epoch 4/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.6381 - loss: 0.6050 - val_accuracy: 0.8002 - val_loss: 0.4099
Epoch 5/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7005 - loss: 0.5221 - val_accuracy: 0.8338 - val_loss: 0.3599
Epoch 6/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.6962 - loss: 0.4979 - val_accuracy: 0.8593 - val_loss: 0.3320
Epoch 7/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7043 - loss: 0.4651 - val_accuracy: 0.8919 - val_loss: 0.2744
Epoch 8/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7455 - loss: 0.4723 - val_accu

In [12]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test_flat, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9071 - loss: 0.1838
Test Loss: 0.1731
Test Accuracy: 0.9174


In [13]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])

# Plot training & validation loss values
plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'])

plt.show()


ModuleNotFoundError: No module named 'matplotlib'